In [42]:
%matplotlib inline
import libs.config
import libs.files as fh
import libs.pipeline as pipe
import libs.parse as p
import libs.twokenize as ark
import libs.resources as r
import numpy as np
import pandas as pd
import unidecode

import codecs

SemEval 2014, Task9, subtask B. There was some error when I downloaded my files from the 2014 test set. Will fix the output file mixing the original file and the file I have downloaded already. It takes too much time to do the re-downloading of the tweets

In [9]:
reload(fh)

<module 'libs.files' from 'libs/files.pyc'>

In [118]:
# read the train file
tw_downloaded = "../1-Input/2014/SemEval2014-task9-test-B-gold.txt.error"
original_file = "../1-Input/2014/SemEval2014-task9-test-B-gold-NEED-TWEET-DOWNLOAD.txt"
final_file = "../1-Input/2014/SemEval2014-task9-test-B-gold.final.txt"
gold_file = "../7-SemEval/2014/SemEval2014-task9-test-B-gold.txt"

# the task in SEMEVAL
file_type = 'B'

In [15]:
# read the problematic
problem_data = {}
with codecs.open(tw_downloaded, 'r',  encoding='utf-8', errors='ignore') as f:
    for line in f:
        data = line.split('\t')
        
        if 'T' in data[1]:
            # it is a Tweet
            text = data[3].replace('\r', '').replace('\n', '')
            try:
                text = text.decode('unicode-escape')
                text = unidecode.unidecode(text)
            except:
                text = text
            problem_data['{}-{}'.format(data[0], data[1])] = {'sent': data[2], 'text':text}

In [17]:
# create a final corrected file
final_data = []
with codecs.open(original_file, 'r',  encoding='utf-8', errors='ignore') as f:
    for line in f:
        data = line.split('\t')
        if len(data)>3:
            # it is NOT a Tweet
            text = data[3].replace('\r', '').replace('\n', '')
            try:
                text = text.decode('unicode-escape')
                text = unidecode.unidecode(text)
            except:
                text = text
            info = {'sent': data[2], 'text':text}
        else:
            info = problem_data['{}-{}'.format(data[0], data[1])]
            
        data = '\t'.join(data[:2] + [info['sent'], info['text']])
        final_data.append(data)           
        
with codecs.open(final_file, 'w', 'utf-8') as f:
    for line in final_data:
        f.write("%s\n" % line)

In [30]:
# read the file
test_feat, gold, test_tweets = pipe.create_features(final_file, file_type, '')
len(test_tweets)

Read 8986 rows from ../1-Input/2014/SemEval2014-task9-test-B-gold.final.txt


7909

In [31]:
# read the information
OUTPUT_DIR = '../3-Output/'
MODEL_OUT_DIR = '../6-Models'
CREATE_TOKENS_FILES = True
PROCESS_DIR = '../2-Processed'
rnd = 9000

# read the config file
cfg = fh.read_config_file("all.yaml")

# read the train file
file_name = "../1-Input/trainingData-B.tsv"
file_type = 'B'
train, labels, train_tweets = pipe.create_features(file_name, file_type, cfg)

# read the dev file
file_name = "../1-Input/devData-B.tsv"
file_type = 'B'
dev, dev_labels, dev_tweets = pipe.create_features(file_name, file_type, cfg)

Read 9683 rows from ../1-Input/trainingData-B.tsv
Read 1653 rows from ../1-Input/devData-B.tsv


In [32]:
# random seed for all the operations
rnd_seed = 9000

# join datasets
train_final = train_tweets[:]
train_final.extend(dev_tweets)

# convert the list into and array that can be indexed
train_labels = np.concatenate((np.array(labels),np.array(dev_labels)))
gold = np.array(gold)

# create the cleaned the tweets
train_clean, test_clean, vect = pipe.create_count_vec(train_final, test_tweets, tokenizer=pipe.tokenize_clean_raw, stop_words=pipe.stop_words)
train_data, test_data, _ = pipe.auto_select_features(pipe.chi2, 5, train_clean, train_labels, test_clean, gold)

train shape: (9576, 19847)
dev shape: (7909, 19847)
Final shape (9576, 993) (7909, 993)


In [34]:
# using negated tokens
train_neg_tokens = [pipe.tokenize_negate_clean_raw(t) for t in train_final]
test_neg_tokens = [pipe.tokenize_negate_clean_raw(t) for t in test_tweets]

# not using negated tokens
train_tokens = [pipe.tokenize_clean_raw(t) for t in train_final]
test_tokens = [pipe.tokenize_clean_raw(t) for t in test_tweets]

In [38]:
def decode_gene(ind):
    """
    Decode a gene to a information that can be processed.
    """
    return [r.lexs[idx] for idx, present in enumerate(ind) if present]

def select_features(lex, train_feat, labels, test_feat):
    """
    Select the most important features according to a given criteria.
    """
    tmp_train_feat, tmp_test_feat, _, _ = pipe.create_lex_vec(train_feat, test_feat)
    tmp_train_feat, tmp_test_feat, selector = pipe.auto_select_features(pipe.mutual_info_classif, lex.selection_percent, 
                                                                       tmp_train_feat, labels, 
                                                                       tmp_test_feat, None)
    return tmp_train_feat, tmp_test_feat

def create_ind_features(ind, base_train, base_test, train_tokens, test_tokens):
    """
    Given an individual, create the features according to its genes
    Params:
        ind: individual generated in the genetic algorithm
    Returns:
        train and test data as a scipy sparse matrix
    """
     # remove the last locus as it refers to the LIWC
    LIWC_gene = ind[-1]
    tmp_ind = ind[:-1]
    
    # load the lexicons according to the gens
    ind_lexs = decode_gene(tmp_ind)
    
    # for each of the lexicons, merge them
    final_train = base_train.copy()
    final_test = base_test.copy()

    for lex in ind_lexs:
#         try:
        print('Joining the lex {}'.format(lex.prefix))
        # create the features from the datasets tokens
        if False:
            train_feat = lex.process_lex(train_neg_tokens, use_best_features=True)
            test_feat = lex.process_lex(test_neg_tokens, use_best_features=True)
        else:
            train_feat = lex.process_lex(train_tokens, use_best_features=True)
            test_feat = lex.process_lex(test_tokens, use_best_features=True)

#         print len(train_feat), len(test_feat)
#             print  test_feat[:3]

        # select the top percent features if this is the case. Set the flag to create vector at save if selection occurs
        if lex.selection_percent:
            train_feat, test_feat = select_features(lex, train_feat, train_labels, test_feat)
        else:
            train_feat, test_feat, _,_ = pipe.create_lex_vec(train_feat, test_feat)

#         print train_data.shape, test_data.shape
        final_train, final_test = pipe.join_lex_features(final_train, train_feat, 
                                                         final_test, test_feat, 
                                                         verbose=True, create_vec=False)
#         except:
#             u.print_exception()
#             print 'error loading ind', ind
#             print lex

        # check if should add LIWC
    if LIWC_gene:
        final_train, final_test = pipe.join_lex_features(final_train, full_train_liwc, 
                                                        final_test, test_liwc, 
                                                        verbose=False, create_vec=True)   
        print final_train.shape, final_test.shape
    return final_train, final_test

In [43]:
# read best gene
df = pd.read_csv('../3-Output/population.csv', header=None, sep=';')
df.columns = ['Gene', 'Stats']

# create the best gene combination
best_individual = eval(df.iloc[0]['Gene'])
X_train, X_test = create_ind_features(best_individual, 
                                      train_data, test_data, 
                                      train_neg_tokens, test_neg_tokens)

Joining the lex BING
train data, lex and final shape:  (9576, 993) (9576, 7) (9576, 1000)
test data, lex and final shape:  (7909, 993) (7909, 7) (7909, 1000)
Joining the lex SWN
train data, lex and final shape:  (9576, 1000) (9576, 8) (9576, 1008)
test data, lex and final shape:  (7909, 1000) (7909, 8) (7909, 1008)
Joining the lex MSOL
Final shape (9576, 413) (7909, 413)
train data, lex and final shape:  (9576, 1008) (9576, 413) (9576, 1421)
test data, lex and final shape:  (7909, 1008) (7909, 413) (7909, 1421)
Joining the lex NRCHASH
train data, lex and final shape:  (9576, 1421) (9576, 7) (9576, 1428)
test data, lex and final shape:  (7909, 1421) (7909, 7) (7909, 1428)
Joining the lex TSLEX
train data, lex and final shape:  (9576, 1428) (9576, 7) (9576, 1435)
test data, lex and final shape:  (7909, 1428) (7909, 7) (7909, 1435)
Joining the lex WNA
Final shape (9576, 111) (7909, 111)
train data, lex and final shape:  (9576, 1435) (9576, 111) (9576, 1546)
test data, lex and final shape:

In [44]:
final_val_scores = pipe.run_multiple_class(X_train, train_labels, X_test, gold, 
                                           rnd_seed=rnd_seed, use_best_params=True,
                                           test_name='test')

In [48]:
final_scores = pipe.predict_test_multi_proc('LogisticRegression', X_train, train_labels, X_test, gold, test_name='test', rnd_seed=rnd)

/Users/adrianow/Documents/Programs/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


In [49]:
final_scores

,train score,test score,f1_test_neg,f1_test_neu,f1_test_pos,train std,test std,f1_test_neg std,f1_test_neu std,f1_test_pos std
LogisticRegression,0.75076,0.67038,0.62608,0.747834,0.714679,0.000136,0.000226,0.00031,0.000123,0.000191


In [52]:
# create the output
cls = pipe.LogisticRegression(**pipe.lr_params)
cls.set_params(random_state=rnd_seed)
cls.fit(X_train, train_labels)
pred_test = cls.predict(X_test)

In [89]:
decode = {
    -1: "negative",
    0:"neutral",
    1: "positive"
}
pred_test_labels = [ decode[val] for val in pred_test]

### Generate final file for submission
The final submission has a specific format that needs to be followed so that it can be properly evaluated. As some tweets were not available, the prediction is made as neutral

In [109]:
# create dictionary with all the key/answers pair as we have missing text in the output
ans_dict = {}
for tf,pred in zip(test_feat, pred_test_labels):
    ans_dict[u"{}-{}".format(tf.sid, tf.uid)] = pred
    
# create the final submission files
out_text = []
with codecs.open(original_file, 'r',  encoding='utf-8', errors='ignore') as f:
    for i, out in enumerate(f):
        data = out.split('\t')
        if len(data)==3:
            sid, uid, sent = data
        else:
            sid, uid, sent, _ = data
        text = "NA\t{}\t{}\n".format(i+1, ans_dict.get(u"{}-{}".format(sid, uid), 'neutral'))
        out_text.append(text)

In [111]:
out_file = '../3-Output/prediction_2014.csv'
with codecs.open(out_file, 'w', 'utf-8') as f:
    f.writelines(out_text)

The final submission scored by SemEval 2014 script was :

| LiveJournal2014	| 	SMS2013	| 	Twitter2013	| 	Twitter2014	| 	Twitter2014Sarcasm	| 
|-------------------------------|--------|-------|-------|-------|
|71.11  | 65.89     | 58.51     | 56.21     |  42.05     |



Not a very good score.

### Disconsidering the failed tweets
Redoing the calculus, using only the information that was able to be downloaded.

In [141]:
orig_data = []
decode = {
    "negative":-1,
    "neutral":0,
    "positive":1,
    None:None
}
# get the dataset for each of the tweets
with codecs.open(original_file, 'r',  encoding='utf-8', errors='ignore') as f:
    with codecs.open(gold_file, 'r',  encoding='utf-8', errors='ignore') as f1:
        for orig, gold in zip(f, f1):
            data_orig = orig.strip().split('\t')
            data_gold = gold.strip().split('\t')
            if len(data_orig)==3:
                sid, uid, sent = data_orig
                text = ''
            else:
                sid, uid, sent, text = data_orig
            
            orig_data.append([data_gold[1], sid, uid, text, decode[sent], decode[ans_dict.get("{}-{}".format(sid,uid), None)] ])

In [142]:
res_df = pd.DataFrame(data = orig_data, columns=['DataSet', 'sid', 'uid', 'text', 'gold_sent', 'pred_sent'])
res_df.head()

,DataSet,sid,uid,text,gold_sent,pred_sent
0,Twitter2014,249298288367525888,T14114531,,1,NaN
1,Twitter2013,189963607449141249,T13118198,,0,0.0
2,Twitter2014,282031301962395648,T14111200,,0,1.0
3,SMS2013,11975,SM112166,Yar he quite clever but aft many guesses lor. ...,-1,1.0
4,LiveJournal2014,136592,LJ112295,Yeah we have Thin Lizzy here I HATE the inform...,-1,-1.0


In [152]:
ret = {}
for db in res_df.DataSet.unique():
    ds_df = res_df[ (res_df.DataSet==db) & (~res_df.pred_sent.isnull())]
    ret[db] = [ '{0:0.2f}'.format(100.0*pipe.score_func(ds_df.gold_sent, ds_df.pred_sent))]
pd.DataFrame(ret)    

,LiveJournal2014,SMS2013,Twitter2013,Twitter2014,Twitter2014Sarcasm
0,71.11,65.89,67.12,64.07,42.05
